Дан вектор целей с очень несбалансированными классами.


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

In [ ]:
iris = load_iris()

features = iris.data

target = iris.target

# Remove first 40 data points
features = features[40:, :]
target = target[40:]

In [ ]:
# Create binary vector pointing if a class is 0
target = np.where((target == 0), 0, 1)

# Disbalanced target vector
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Выше видно, что наши данные несбаласированы: класса 0 меньше количество, чем класса 1. Для того, чтобы создать работующую модель МО, необходимо предоставить алгоритму достаточное количество данных об всех классов. С этой целю происходит балансировка данных.



---

Многие алгоритмы в scikit-learn предлагают параметр для взвешивания классов во время тренировки, чтобы противодействовать эффекту их разбалансировки. Хотя мы еще его не рассматривали, классификатор на основе случайного леса *RandomForestciassifier* является популярным классификационным алгоритмом и включает параметр class *weight*.

In [ ]:
# Create a classifier based on the random forest with weights
RandomForestClassifier(class_weight={0:.9, 1:.1})
RandomForestClassifier(bootstrap=True,
                       class_weight={0:.9, 1:.1},
                       criterion='gini',
                       max_depth=None,
                       max_features='auto',
                       max_leaf_nodes=None,
                       min_impurity_decrease=0.0,
                       min_samples_leaf=1,
                       min_samples_split=2,
                       min_weight_fraction_leaf=0.0,
                       n_estimators=10,
                       n_jobs=1,
                       oob_score=False,
                       random_state=None,
                       verbose=0,
                       warm_start=False)

RandomForestClassifier(class_weight={0: 0.9, 1: 0.1}, max_features='auto',
                       n_estimators=10, n_jobs=1)

Либо передать аргумент *balanced*, который автоматически создает веса, обратно пропорциональные частотам классов

In [ ]:
RandomForestClassifier(class_weight='balanced')
RandomForestClassifier(bootstrap=True,
                       class_weight='balanced',
                       criterion='gini',
                       max_depth=None,
                       max_features='auto',
                       max_leaf_nodes=None,
                       min_impurity_decrease=0.0,
                       min_samples_leaf=1,
                       min_samples_split=2,
                       min_weight_fraction_leaf=0.0,
                       n_estimators=10,
                       n_jobs=1,
                       oob_score=False,
                       random_state=None,
                       verbose=0,
                       warm_start=False)

RandomForestClassifier(class_weight='balanced', max_features='auto',
                       n_estimators=10, n_jobs=1)

В качестве альтернативы можно понизить мажоритарный класс или повысить миноритарный класс. В понижающем отборе мы произвольно делаем выборку без возврата из мажоритарного класса (т.е. класса с большим количеством наблюдений), чтобы создать новое подмножество наблюдений, равное по размеру миноритарному классу. Например, если миноритарный класс имеет 10 наблюдений, мы случайным образом отберем 10 наблюдений из мажоритарного класса и используем эти 20 наблюдений в качестве наших данных. Сделаем это, используя наши несбалансированные данные цветков ириса:

In [ ]:
i_class0 = np.where(target == 0)[0]  # returns indexes of every 0
i_class1 = np.where(target == 1)[0]  # returns indexes of every 0

In [ ]:
n_class0 = len(i_class0)
n_class1 = len(i_class1)

In [ ]:
# Для каждого наблюдения класса 0 сделать случайную выборку
# из класса 1 без возврата
i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace=False)

In [ ]:
# Соединить вектор целей класса 0 с
# вектором целей пониженного класса 1
np.hstack((target[i_class0], target[i_class1_downsampled]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
# Соединить матрицу признаков класса 0 с
# матрицей признаков пониженного класса 1
np.vstack((features[i_class0, :], features[i_class1_downsampled, :]))

array([[5. , 3.5, 1.3, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.9, 0.4],
       [4.8, 3. , 1.4, 0.3],
       [5.1, 3.8, 1.6, 0.2],
       [4.6, 3.2, 1.4, 0.2],
       [5.3, 3.7, 1.5, 0.2],
       [5. , 3.3, 1.4, 0.2],
       [7.7, 3.8, 6.7, 2.2],
       [6.2, 2.9, 4.3, 1.3],
       [5.6, 3. , 4.5, 1.5],
       [6.4, 2.7, 5.3, 1.9],
       [6. , 3.4, 4.5, 1.6],
       [6.7, 3. , 5.2, 2.3],
       [5.6, 2.7, 4.2, 1.3],
       [6.8, 3.2, 5.9, 2.3],
       [6.4, 3.2, 4.5, 1.5],
       [6.5, 3. , 5.8, 2.2]])

Другой вариант — повысить миноритарный класс. В повышающем отборе для каждого наблюдения в мажоритарном классе мы случайно отбираем наблюдение из миноритарного класса с возвратом. Конечным результатом является одинаковое количество наблюдений из миноритарных и мажоритарных классов. Повышающий отбор реализуется очень похоже на понижающий отбор, только наоборот:

In [ ]:
# Для каждого наблюдения в классе 1 сделать случайную выборку
# из класса 0 с возвратом
i_class0_upsampled = np.random.choice(i_class0, size=n_class1, replace=True)

In [ ]:
# Соединить повышенный вектор целей класса 0
# с вектором целей класса 1
np.concatenate((target[i_class0_upsampled], target[i_class1]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [ ]:
# Соединить повышенную матрицу признаков класса 0
# с матрицей признаков класса 1
np.vstack((features[i_class0_upsampled, :], features[i_class1, :]))

array([[5. , 3.5, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [4.6, 3.2, 1.4, 0.2],
       [5.3, 3.7, 1.5, 0.2],
       [5. , 3.3, 1.4, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.6, 0.2],
       [5.3, 3.7, 1.5, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5. , 3.3, 1.4, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [4.5, 2.3, 1.3, 0.3],
       [5. , 3.3, 1.4, 0.2],
       [5. , 3.5, 1.3, 0.3],
       [5. , 3.5, 1.3, 0.3],
       [4.8, 3. , 1.4, 0.3],
       [5. , 3.5, 1.6, 0.6],
       [4.8, 3. , 1.4, 0.3],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.9, 0.4],
       [5. , 3.5, 1.6, 0.6],
       [4.6, 3.2, 1.4, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [4.8, 3. , 1.4, 0.3],
       [5.1, 3.8, 1.6, 0.2],
       [5.3, 3.7, 1.5, 0.2],
       [5.3, 3.7, 1.5, 0.2],
       [4.5, 2.3, 1.3, 0.3],
       [5.3, 3.7, 1.5, 0.2],
       [5. , 3.5, 1.3, 0.3],
       [5. , 3.3, 1.4, 0.2],
       [5. , 3

В реальном мире несбалансированные классы встречаются повсюду — большинство посетителей не нажимают кнопку Купить, и многие виды рака, к счастью, — редки. По этой причине обработка несбалансированных классов является в машинном самообучении распространенной задачей.